In [1]:
import json

with open('../../anime-offline-database/anime-offline-database.json') as f:
    manami = json.load(f)

In [75]:
len(manami['data'])

30472

In [76]:
mangaki = Work.objects.filter(category__slug='anime')
mangaki.count()  # Sur Mangaki

10987

In [77]:
manami['data'][42]

{'sources': ['https://anidb.net/anime/8606'],
 'type': 'Movie',
 'title': '.hack//Sekai no Mukou ni',
 'picture': 'https://cdn.anidb.net/images/main/221181.jpg',
 'relations': ['https://anidb.net/anime/7883'],
 'thumbnail': 'https://img7.anidb.net/pics/anime/thumbs/50x65/221181.jpg-thumb.jpg',
 'episodes': 1,
 'synonyms': ['.hack//Beyond the World',
  '.hack//The Movie',
  '.hack//Versus: The Thanatos Report',
  '.злам// Поза «World`-ом»',
  'Злам - Поза Світом',
  'ドット ハック',
  'ドット ハック セカイの向こうに',
  'ドットハック セカイの向こうに']}

In [85]:
from collections import defaultdict
import re

references = set()
url1 = defaultdict(list)

for manami_id, entry in enumerate(manami['data']):
    references.add('manami/{}'.format(manami_id))
    for url in entry['sources']:
        ref = re.sub(r'https?://', '', url)
        references.add(ref)
        url1[manami_id].append(ref)
len(references)

84255

In [86]:
url2 = defaultdict(set)

anidb_id = dict(Work.objects.filter(anidb_aid__gt=0).values_list('id', 'anidb_aid'))
for mangaki_id, aid in anidb_id.items():
    ref1 = 'mangaki.fr/work/{}'.format(mangaki_id)
    ref2 = 'anidb.net/anime/{}'.format(aid)
    references.update([ref1, ref2])
    url2[mangaki_id].update([ref1, ref2])

for mangaki_id, url in Reference.objects.values_list('work_id', 'url'):
    ref1 = 'mangaki.fr/work/{}'.format(mangaki_id)
    ref2 = re.sub(r'https?://', '', url).replace('anidb.net/perl-bin/animedb.pl?show=anime&aid=', 'anidb.net/anime/')
    references.update([ref1, ref2])
    url2[mangaki_id].update([ref1, ref2])
len(references)

97013

In [87]:
ids = dict(zip(sorted(references), range(len(references))))

In [88]:
list(ids.keys())[:5]

['anidb.net/anime/1',
 'anidb.net/anime/10',
 'anidb.net/anime/100',
 'anidb.net/anime/1000',
 'anidb.net/anime/10000']

In [89]:
from tryalgo.kruskal import UnionFind

uf = UnionFind(len(references))
nb_merge = 0
for manami_id, refs in url1.items():
    for ref in refs:
        nb_merge += uf.union(ids[ref], ids['manami/{}'.format(manami_id)])
nb_merge

53783

In [90]:
nb_has_anidb = 0
for mangaki_id, refs in url2.items():
    has_anidb = False
    for ref in refs:
        if 'anidb' in ref:
            has_anidb = True
        nb_merge += uf.union(ids[ref], ids['mangaki.fr/work/{}'.format(mangaki_id)])
    nb_has_anidb += has_anidb
nb_merge, '{}/{} has AniDB reference'.format(nb_has_anidb, len(url2))

(65980, '313/11967 has AniDB reference')

In [91]:
313/11967

0.026155260299156012

In [92]:
clusters = defaultdict(list)
for ref, ref_id in ids.items():
    clusters[uf.find(ref_id)].append(ref)
len(clusters)

31033

In [103]:
from collections import Counter

c = Counter()
nb_cdup_mangaki = 0
nb_cdup_manami = 0
total_ref = 0
for cluster in clusters.values():
    nb_mangaki = len([ref for ref in cluster if ref.startswith('mangaki')])
    nb_manami = len([ref for ref in cluster if ref.startswith('manami')])
    if nb_mangaki > 0:
        total_ref += len(cluster) - nb_mangaki - nb_manami
    if nb_mangaki >= 2 and nb_manami >= 1:
        nb_cdup_mangaki += 1
    if nb_manami >= 2 and nb_mangaki >= 1:
        nb_cdup_manami += 1
    # if nb_mangaki + nb_manami >= 14:
    #     print(cluster)  # It was Kara no Kyoukai
    c[nb_mangaki, nb_manami] += 1
c, nb_cdup_mangaki, nb_cdup_manami, total_ref

(Counter({(0, 1): 19856,
          (1, 1): 9676,
          (1, 2): 97,
          (2, 1): 554,
          (3, 2): 12,
          (2, 2): 41,
          (3, 1): 47,
          (8, 7): 1,
          (3, 3): 2,
          (5, 3): 1,
          (4, 2): 1,
          (8, 4): 1,
          (2, 3): 1,
          (4, 1): 13,
          (5, 1): 1,
          (1, 0): 721,
          (2, 0): 8}),
 675,
 157,
 27293)

In [101]:
from collections import Counter

c2 = Counter()
nb_with_anidb = 0
nb_c_mangaki = 0
for cluster in clusters.values():
    nb_mangaki = len([ref for ref in cluster if ref.startswith('mangaki')])
    nb_anidb = len([ref for ref in cluster if ref.startswith('anidb')])
    if nb_mangaki > 0:
        nb_c_mangaki += 1
        if nb_anidb > 0:
            nb_with_anidb += 1
    c2[nb_mangaki, nb_anidb] += 1
c2, '{}/{} désormais'.format(nb_with_anidb, nb_c_mangaki)

(Counter({(0, 1): 8924,
          (1, 1): 1836,
          (2, 1): 81,
          (0, 4): 1,
          (3, 1): 19,
          (8, 1): 2,
          (5, 1): 1,
          (4, 1): 1,
          (0, 0): 10931,
          (1, 0): 8658,
          (2, 0): 523,
          (3, 0): 42,
          (4, 0): 13,
          (5, 0): 1}),
 '1940/11177 désormais')

In [102]:
1940/11177

0.17357072559720854

In [104]:
Reference.objects.count()

12016